### Sequence Analysis

The objective of this notebook is to
1. Create sequences from trajectories for customers
2. Load sequences into R package TraMineR
3. Use TraMineR functions to calculate distance between sequences using TRATE or 2-1-1 costs
4. Cluster the distance matrix using PAM/k-medoids clustering
5. Plot medoid trajectories


In [ ]:
import random
import sys
sys.path.append("..")

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import rpy2
import datetime
from dateutil.relativedelta import relativedelta
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import seaborn as sns
from shapely.geometry import Point
get_ipython().magic(u'matplotlib inline')
%matplotlib inline


from connect_db import db_connection

# filter annyoing warning from pandas
import warnings
warnings.filterwarnings('ignore')

In [ ]:
username='kmohan'

In [ ]:
cred_location = '/mnt/data/'+username+'/utils/data_creds_redshift.json.nogit'
db = db_connection.DBConnection(cred_location)

In [ ]:
# query for the 1week sample

query = """
select customer_nr, com_locs_new as locations,times_new as times, st_time,en_time, mcc 
from tuscany.customer_arrays
where times_new is not null
and st_time >= '2017-06-01 00:00:00' and st_time < '2017-09-01 00:00:00'
and mcc = 262
"""
# drop 'customer_id' to save memory
df_trips = db.sql_query_to_data_frame(query, cust_id=True)

In [ ]:
df_trips.head()

In [ ]:
df_trips.shape

In [ ]:
max_time = max(df_trips['en_time'] - df_trips['st_time'])

In [ ]:
import math
math.ceil(max_time.total_seconds()/(60*60*12))

In [ ]:
max_time = max(df_trips['en_time'] - df_trips['st_time'])
ncols = math.ceil(max_time.total_seconds()/(60*60*12))
columns = np.linspace(1,ncols,ncols)

In [ ]:
len(columns)

In [ ]:
n_cus = df_trips.shape[0]

In [ ]:
len(df_trips)

In [ ]:
df_trips['times'] = df_trips['times'].str.split(', ').tolist()
df_trips['locations'] = df_trips['locations'].str.split(', ').tolist()

In [ ]:
df_trips.head()

In [ ]:
def len_array(row):
    return len(row['locations'])

def len_times(row):
    return len(row['times'])


In [ ]:
cus_nr = df_trips['customer_nr']

In [ ]:
com_lens = df_trips.apply(len_array,axis=1)
time_lens = df_trips.apply(len_times,axis=1)
cus_nr = df_trips['customer_nr']

df_lens = pd.DataFrame({'customer_nr':cus_nr,'com_len' : com_lens, 'time_len': time_lens})

In [ ]:
df_lens.head()

Monday 0 to Sunday 6

In [ ]:
df_lens['diff'] = df_lens['com_len'] - df_lens['time_len']

In [ ]:
filt_cus = df_lens[df_lens['diff'] != 1]['customer_nr']

In [ ]:
len(filt_cus)


In [ ]:
weeks = 10*np.linspace(1,6,6)
days= np.linspace(0,6.5,14)

In [ ]:
weeks

In [ ]:
columns = np.add.outer(weeks,days).flatten()

In [ ]:
len(columns)

In [ ]:
df_sequence = pd.DataFrame(columns=columns,index=cus_nr)

In [ ]:
df_sequence.head()

In [ ]:
def location_with_max_time(array_like):
    return np.bincount(array_like).argmax()

In [ ]:
for i in range(0,len(cus_nr)):
#    print(i)
    df_row = df_trips[i:i+1]
    cus = df_row['customer_nr'][i]
    st_wk = df_row['st_time'][i].weekday()
    st_hr = df_row['st_time'][i].hour
    country = df_row['mcc'][i]
    # Initialising all values on the sequence to be country of origin
    df_sequence.loc[[cus],columns[:]] = country
    times = [0]
    minutes = [0]
    timestamps = [df_row['st_time'][i]]
    cum_mins = np.cumsum(np.array(list(map(int,df_row['times'][i]))))
    minutes.extend(np.array(list(map(int,df_row['times'][i]))))
    times.extend(np.cumsum(np.array(list(map(int,df_row['times'][i])))))
    timestamps.extend(np.datetime64(df_row['st_time'][i]) + cum_mins.astype('timedelta64[m]'))
    coms = np.array(df_row['com_locs'][i])
    tmp_df = pd.DataFrame({'Minutes' : minutes, 'Mins_cum' : times, 'Locations': coms, 'Timestamps':timestamps })
    tmp_df['date'] = tmp_df['Timestamps'].dt.date
    ts = pd.Series(coms,dtype=np.int64,index=tmp_df['Timestamps'])
    ts = ts.resample('1T').bfill()
    ts2 = ts.resample('12H').apply(location_with_max_time)
    col_idx_st = 2*st_wk + int(st_hr/12)
    df_sequence.loc[[cus],columns[col_idx_st:(col_idx_st + len(ts2))]] = ts2.values

In [ ]:
df_sequence.loc[:,columns[74:84]].sort_values(by=[62],ascending=0).head()

In [ ]:
df_sequence_trimmed = df_sequence.loc[:,columns[0:73]]

In [ ]:
!pwd

In [ ]:
df_sequence.to_csv("../../sequences_Tuscany_Aug_20000_sample.csv")

In [ ]:
st = df_trips['st_time'][0]

In [ ]:
st = np.datetime64(st)

In [ ]:
cum_array0 = cum_array0.astype('timedelta64[m]')


In [ ]:
st + cum_array0

In [ ]:
df_trips.tail()

In [ ]:
df_row = df_trips[873:874]

In [ ]:
df_row

In [ ]:
df_row['st_time'][873]

In [ ]:
int(df_row['st_time'][873].hour/6)

In [ ]:
cus = df_row['customer_nr'][873]
st_wk = df_row['st_time'][873].weekday()
country = df_row['mcc'][873]
times = [0]
minutes = [0]
timestamps = [df_row['st_time'][873]]
cum_mins = np.cumsum(np.array(list(map(int,df_row['times'][873]))))
minutes.extend(np.array(list(map(int,df_row['times'][873]))))
times.extend(np.cumsum(np.array(list(map(int,df_row['times'][873])))))
timestamps.extend(np.datetime64(df_row['st_time'][873]) + cum_mins.astype('timedelta64[m]'))
coms = np.array(df_row['com_locs'][873])
tmp_df = pd.DataFrame({'Minutes' : minutes, 'Mins_cum' : times, 'Locations': coms, 'Timestamps':timestamps })

In [ ]:
cus

In [ ]:
tmp_df['date'] = tmp_df['Timestamps'].dt.date

In [ ]:
tmp_df.head()

In [ ]:
ts = pd.Series(coms,dtype=np.int64,index=tmp_df['Timestamps'])

In [ ]:
ts.head()

In [ ]:
ts = ts.resample('1T').bfill()

In [ ]:
ts['2017-08-12']

In [ ]:
def location_with_max_time(array_like):
    return np.bincount(array_like).argmax()

In [ ]:
ts2 = ts.resample('12H').apply(location_with_max_time)

In [ ]:
ts2

In [ ]:
len(columns)

In [ ]:
st_hr = df_row['st_time'][0].hour

In [ ]:
2*st_wk + 1 if st_hr >=12 else 0

In [ ]:
len(ts2)

In [ ]:
df_sequence.shape

In [ ]:
len(columns[13:(13+59)])

In [ ]:
df_sequence.loc[[cus],columns[:]] = 100

In [ ]:
df_sequence.loc[[cus],columns[:]]

In [ ]:
counts

In [ ]:
str(12)+'H'

In [ ]:
def str_list_to_int_list(array_like):
    return list(map(int,array_like))

In [ ]:
def str_to_list(df_trips):
    """
    Convert a str (output of the SQL query) into a list of strs

    Parameters:
    df_trips: DataFrame containing the column 'locations' and 'times'
    """

    # replace str of geolocations by a list of strs with geolocation codes
    df_trips['locations'] = list(map(str_list_to_int_list,df_trips['locations'].str.split(', ').tolist()))

    # replace str of geolocations by a list of strs with geolocation codes
    df_trips['times'] = list(map(str_list_to_int_list,df_trips['times'].str.split(', ').tolist()))

In [ ]:
str_to_list(df_trips)

In [ ]:
df_trips.head()

In [ ]:
def location_with_max_time(array_like):
    """
    Returns the value in array apprearing most frequently

    Parameters:
    array_like: any array
    """
    return np.bincount(array_like).argmax()

In [ ]:
def create_sequence_for_individual(i,align_by_day_of_week,window_hrs,country_for_missing,ncols):
    # extracting the row for each customer
    df_row = df_trips[i:i+1]
    cus = int(df_row['customer_nr'][i])
    st_wk = df_row['st_time'][i].weekday()
    st_hr = df_row['st_time'][i].hour
    country = int(df_row['mcc'][i])

    seq = [np.nan] * ncols
    seq[0] = cus
    # Initialising all values on the sequence to be country of origin if set to True
    if country_for_missing == True:
        seq[1:] = [country]*(ncols-1)

    # Creating the Pandas Series object from the list of 'times'
    # Initialising the time array
    timestamps = [np.datetime64(df_row['st_time'][i])]

    # Cummulating times spent at each location to create timestamps
    cum_mins = np.cumsum(np.array(list(map(int,df_row['times'][i]))))
    timestamps.extend(np.datetime64(df_row['st_time'][i]) + cum_mins.astype('timedelta64[m]'))

    # Getting list of locations
    locs = np.array(df_row['locations'][i])
    # Defining the Pandas Series object
    ts = pd.Series(locs,dtype=np.int64,index=timestamps)

    # Resampling sequence for the required window size
    # 1 minute resolution before resampling to window_hrs as we want to find the location spent maximum time at in the window
    ts = ts.resample('1T').bfill()
    # Resampling to window_hrs
    ts2 = ts.resample(str(window_hrs)+'H').apply(location_with_max_time)

    # Identifying the columns to insert into
    if align_by_day_of_week == True:
        col_idx_st = int(24/window_hrs)*st_wk + int(st_hr/window_hrs)
    else:
        col_idx_st = int(st_hr/window_hrs)

    # Inserting location values into the sequence dataframe
    # df_sequence.loc[[cus],columns[col_idx_st:(col_idx_st + len(ts2))]] = ts2.values
    seq[col_idx_st:(col_idx_st + len(ts2))] = ts2.values

    return seq

# TODO: Save function

In [ ]:
def create_sequences(df_trips,align_by_day_of_week=True,window_hrs=3,country_for_missing=True,n_threads=5):
    """
    Create a dataframe of aligned sequences for sequence clustering analysis

    Parameters:
    df_trips: DataFrame containing the column 'locations', 'times', 'st_time','customer_nr','mcc'
    align_by_day_of_week: If True, the sequences are aligned by the day of week of arrival. Else, the sequences are aligned 
        by their respective first day of arrival.
    window_hrs: window size for sequence creation in hours. A sequence would contain a location for every 'window_hrs' 
        from start to end times
    country_for_missing: If True, the location for entries in the sequence when the individual wasn't in Italy would be
        set to the MCC code of the respective country
    n_threads: Number of threads to use in parallel
    """    

    # importing math for ceiling function
    import math
    from multiprocessing import Pool
    from itertools import repeat
    
    # finding the maximum time spent by an individual to set the length fo sequence
    max_time = max(df_trips['en_time'] - df_trips['st_time'])
    ncols = math.ceil(max_time.total_seconds()/(60*60*window_hrs))

    # If aligning by day of week of arrival, we need additional columns 
    # as someone could arrive on a sunday and stay for max_time
    if align_by_day_of_week == True:
        ncols += 6*math.ceil(24/window_hrs)

    # weeks = 10*np.linspace(1,6,6)
    # days= np.linspace(0,6.5,14)
    # columns = np.add.outer(weeks,days).flatten()
    
    # Initialising the sequence dataframe with NAs
    columns = np.linspace(1,ncols,ncols)
    cus_nr = df_trips['customer_nr']
    # df_sequence = pd.DataFrame(np.nan,columns=columns,index=cus_nr)

    # Looping through every customer array to create the aligned sequences data frame
    p = Pool(n_threads)
    l = [i for i in range(0,len(cus_nr))]
    sequences_as_lists = p.starmap(create_sequence_for_individual, zip(l, [align_by_day_of_week]*len(cus_nr), [window_hrs]*len(cus_nr), [country_for_missing]*len(cus_nr), [ncols+1]*len(cus_nr)))

    # Converting list of lists into a dataframe
    col_names = ['customer_nr'] + list(map(str,np.linspace(1,ncols,ncols)))
    df_sequence = pd.DataFrame.from_records(sequences_as_lists,columns=col_names)

    # Setting customer number to be the index
    df_sequence = df_sequence.set_index('customer_nr')
    return df_sequence

In [ ]:
sd_seq1 = create_sequences(df_trips,align_by_day_of_week=False,window_hrs=12,country_for_missing=True,n_threads=12)

In [ ]:
sd_seq1.head()

In [ ]:
sd_seq1.to_csv("/mnt/data/kmohan/sequences_Germans_Summer.csv")

### Clustering Sequences using TraMineR package in R

 - loading dataframe saved above into R environment
 - creating sequence object
 - substitution cost using TRATE
 - distance between sequences using OMA
 - clustering using PAM/k-medoids

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
#install.packages('TraMineR')
install.packages('TraMineRextras')
install.packages('fpc')
install.packages('WeightedCluster')
install.packages('foreach')
install.packages('parallel')
install.packages('doParallel')


In [ ]:
%%R
library(cluster)
library(lattice)
library(TraMineR)
library(TraMineRextras)
library(fpc)
library(foreach)
library(parallel)
library(doParallel)
library(WeightedCluster)


In [ ]:
%%R
df_seq <- read.csv("../../sequences_Germans_Summer.csv")
agg_seq <- wcAggregateCases(df_seq[,-1])
print(agg_seq)

In [ ]:
%%R
unique_seq <- df_seq[agg_seq$aggIndex, -1]
seq_obj <- seqdef(unique_seq, weights = agg_seq$aggWeights)
seq_subcost <- seqcost(seq_obj,method="CONSTANT",cval=2)

In [ ]:
%%R
seq_dist <- seqdist(seq_obj,method = "OM",sm=seq_subcost$sm,full.matrix=FALSE)

In [ ]:
%%R
seq_dist <- seqdist(seq_obj[2,],refseq=seq_obj[1,],method = "OM",sm=seq_subcost$sm)

In [ ]:
%%R
seq_dist

In [ ]:
%%R
packageVersion("TraMineR")
getRversion()
gc()

In [ ]:
%%R
version

In [ ]:
%%R
load("../../seqdist_Tuscany_Aug_10000_sample.RSav")

In [ ]:
%%R
## Stats to find number of clusters
get_pam_cluster_stats <- function(dist_matrix,k){
  cl <- pam(dist_matrix,k=k,diss=TRUE)
  cs <- cluster.stats(dist_matrix,clustering = cl$clustering)
  return(c(cs$ch,cs$avg.silwidth ))
}

cl.stats = data.frame(matrix(0,nrow=5,ncol=2))

for (i in c(2:10)){
    cl.stats[i,] = get_pam_cluster_stats(seq_dist_trate,i)
}


In [ ]:
%%R
plot(x=c(2:10),y=cl.stats[2:10,1],type='l')


In [ ]:
%%R
clusterpam_trate <- pam(seq_dist_trate, k=4, diss = TRUE)

In [ ]:
%%R
clustering_results = data.frame(ids=df_seq[,1], cluster=clusterpam_trate$clustering)
write.csv(clustering_results,"../../clusters_Tuscany_4_Aug_10000_sample.csv")

In [ ]:
%%R
medoids_cus_ids = df_seq[clusterpam_trate$medoids,1]
write.csv(medoids_cus_ids,"../../medoids_Tuscany_4_Aug_10000_sample.csv")

In [ ]:
%%R
save(seq_dist_trate,file="../../seqdist_Tuscany_Aug_10000_sample.RSav")

In [ ]:
%%R
K<-5
for (k in 1:K){
    seqdplot(seq_obj[clusterpam_trate$clustering==k,])
}


#### Experimenting with nested foreach

In [ ]:
%%R
sim <- function(seq_obj,a,b,sm){
    k <- sum(b<=a)
    if (k < length(b)){
        return (c(rep(NA,k),seqdist(seq_obj[b[(k+1):length(b)],],refseq=seq_obj[a,],method = "OM",sm=sm)))                
    }else{
        return (rep(NA,k))                
    }
    
}

In [ ]:
%%R
no_cores <- 8
registerDoParallel(makeCluster(no_cores))

N <- dim(seq_obj)[1]
G <- 10000
avec <- c(1:N)

seq_dist <- foreach(a=avec, .combine='cbind') %:%
foreach(b=c(1:ceiling(N/G)), .combine='c',.packages = c("TraMineR")) %dopar% {
sim(seq_obj,a,c((G*(b-1)+1):min((G*b),N)),seq_subcost$sm)
}

stopImplicitCluster()


In [ ]:
%%R
dim(seq_obj)

In [ ]:
%%R
simple_sim <- function(a,b){
    return (10*a+b)
}

In [ ]:
%%R
no_cores <- detectCores()
registerDoParallel(makeCluster(no_cores))

bvec <- c(1:10000)
avec <- c(1:1000000)
x <- foreach(a=avec, .combine='c') %dopar% simple_sim(a, 10)

stopImplicitCluster()

In [ ]:
%%R
no_cores <- detectCores()
registerDoParallel(makeCluster(no_cores))
no_cores

### Visualization and descriptives on cluster results

In [ ]:
df_med = pd.read_csv("../../mediods_5_Jul_Aug_10000_sample.csv")

In [ ]:
med_cus_nos = np.array(df_med.iloc[:,1])

In [ ]:
med_cus_nos

In [ ]:
sum(df_trips['customer_nr'] == med_cus_nos[2])

In [ ]:
df_med.merge(df_trips,how='inner', left_on='x', right_on='customer_nr')

In [ ]:
# Load maps data 

# load data from TPT
regions = r"/mnt/data/shared/Boundaries regions and municipalities Italy 2016/Reg2016_WGS84_g/Reg_2016_WGS84_g.shp"
provinces = r"/mnt/data/shared/Boundaries regions and municipalities Italy 2016/CMProv2016_WGS84_g/CMprov2016_WGS84_g.shp"
municipalities = r"/mnt/data/shared/Boundaries regions and municipalities Italy 2016/Com2016_WGS84_g/Com2016_WGS84_g.shp"
new_reg =  r"/mnt/data/shared/ITA_shapefiles/Tus_28districts.shp"

# important cities 
important_cities_file = r"/mnt/data/shared/important_cities.csv"

In [ ]:
gdf_mun = gpd.read_file(municipalities)
gdf_mun['geometry'] = gdf_mun['geometry'].to_crs(epsg=4326)

In [ ]:
def plot_trajectory(list_of_comunes=False):
    '''
    Parameters:
    
        list_of_comunes: list of pro_com (as ints)    
    '''
    
    # comune centroids 
    df_centroids = pd.read_csv(r"/mnt/data/shared/comune_centroids.csv")
        
    fig = plt.figure(figsize=(15, 15))
    ax = plt.subplot(1,1,1)
    
    gdf_mun.plot(ax=ax, color='white', edgecolor='gray', alpha=0.5)
    
    
    if list_of_comunes is not False:
        
        trip = pd.DataFrame(list_of_comunes, columns=['PRO_COM'])
        trip = trip.merge(df_centroids, how='inner', left_on='PRO_COM', right_on='pro_com')
        
    plt.plot(trip['lat'], trip['lon'], '-o')
    plt.axis('off')

In [ ]:
# query for the 1week sample

query = """
select customer_nr,com_locs_new from tuscany.customer_arrays
where customer_nr in (6830799, 5955872, 1179935, 6804043, 1418535)
"""
# drop 'customer_id' to save memory
df_clusters = db.sql_query_to_data_frame(query, cust_id=True)

In [ ]:
df_clusters['com_locs_new'] = df_clusters['com_locs_new'].str.split(', ').tolist()

In [ ]:
for i in range(0,5):
    plot_trajectory(list(map(int,df_clusters.iloc[i,1])))

In [ ]:
# comune centroids 
df_centroids = pd.read_csv(r"/mnt/data/shared/comune_centroids.csv")

fig = plt.figure(figsize=(15, 15))
ax = plt.subplot(1,1,1)

gdf_mun.plot(ax=ax, color='white', edgecolor='gray', alpha=0.5)

trip = pd.DataFrame(list(map(int,df_clusters.iloc[0,1])), columns=['PRO_COM'])
trip = trip.merge(df_centroids, how='inner', left_on='PRO_COM', right_on='pro_com')

plt.plot(trip['lat'], trip['lon'], '-o')

trip = pd.DataFrame(list(map(int,df_clusters.iloc[1,1])), columns=['PRO_COM'])
trip = trip.merge(df_centroids, how='inner', left_on='PRO_COM', right_on='pro_com')

plt.plot(trip['lat'], trip['lon'], '-o')

trip = pd.DataFrame(list(map(int,df_clusters.iloc[2,1])), columns=['PRO_COM'])
trip = trip.merge(df_centroids, how='inner', left_on='PRO_COM', right_on='pro_com')

plt.plot(trip['lat'], trip['lon'], '-o')

trip = pd.DataFrame(list(map(int,df_clusters.iloc[3,1])), columns=['PRO_COM'])
trip = trip.merge(df_centroids, how='inner', left_on='PRO_COM', right_on='pro_com')

plt.plot(trip['lat'], trip['lon'], '-o')

trip = pd.DataFrame(list(map(int,df_clusters.iloc[4,1])), columns=['PRO_COM'])
trip = trip.merge(df_centroids, how='inner', left_on='PRO_COM', right_on='pro_com')

plt.plot(trip['lat'], trip['lon'], '-o')


plt.axis('off')


#### Experimenting with parallel processing below

In [ ]:
from multiprocessing import Pool
from itertools import repeat

X = 0

def f(x, y, z):
    a = x*y*z
    b = x+y+z 
    return list([a,b])

if __name__ == '__main__':
    p = Pool(5)
    l =[i for i in range(0,10)]
    y = p.starmap(f, zip([1,2,3], [4,5,6],[7,8,9]))

In [ ]:
map(,y)

In [ ]:
from itertools import product
names = ['Brown', 'Wilson', 'Bartlett', 'Rivera', 'Molloy', 'Opie']
print(product(names, repeat=2))

In [ ]:
print(pd.DataFrame({'col1': [1], 'col2' : [2]}))

In [ ]:
align_by_day_of_week=True,window_hrs=3,country_for_missing=True,ncols=(df_sequence.shape[1]+1)

In [ ]:
a = [True]*10

In [ ]:
a

In [ ]:
def create_sequence_for_individual(i,align_by_day_of_week,window_hrs,country_for_missing,ncols):
    # extracting the row for each customer
    df_row = df_trips[i:i+1]
    cus = int(df_row['customer_nr'][i])
    st_wk = df_row['st_time'][i].weekday()
    st_hr = df_row['st_time'][i].hour
    country = int(df_row['mcc'][i])

    seq = [np.nan] * ncols
    seq[0] = cus
    # Initialising all values on the sequence to be country of origin if set to True
    if country_for_missing == True:
        seq[1:] = [country] * (ncols-1)

    # Creating the Pandas Series object from the list of 'times'
    # Initialising the time array
    timestamps = [np.datetime64(df_row['st_time'][i])]

    # Cummulating times spent at each location to create timestamps
    cum_mins = np.cumsum(np.array(list(map(int,df_row['times'][i]))))
    timestamps.extend(np.datetime64(df_row['st_time'][i]) + cum_mins.astype('timedelta64[m]'))

    # Getting list of locations
    locs = np.array(df_row['locations'][i])
    # Defining the Pandas Series object
    ts = pd.Series(locs,dtype=np.int64,index=timestamps)

    # Resampling sequence for the required window size
    # 1 minute resolution before resampling to window_hrs as we want to find the location spent maximum time at in the window
    ts = ts.resample('1T').bfill()
    # Resampling to window_hrs
    ts2 = ts.resample(str(window_hrs)+'H').apply(location_with_max_time)

    # Identifying the columns to insert into
    if align_by_day_of_week == True:
        col_idx_st = int(24/window_hrs)*st_wk + int(st_hr/window_hrs)
    else:
        col_idx_st = int(st_hr/window_hrs)

    # Inserting location values into the sequence dataframe
    # df_sequence.loc[[cus],columns[col_idx_st:(col_idx_st + len(ts2))]] = ts2.values
    seq[col_idx_st:(col_idx_st + len(ts2))] = ts2.values

    return seq

In [ ]:
def create_sequences(df_trips,align_by_day_of_week=True,window_hrs=3,country_for_missing=True,n_threads=5):
    """
    Convert a str (output of the SQL query) into a list of strs

    Parameters:
    df_trips: DataFrame containing the column 'locations', 'times', 'st_time','customer_nr','mcc'
    align_by_day_of_week: If True, the sequences are aligned by the day of week of arrival. Else, the sequences are aligned 
        by their respective first day of arrival.
    window_hrs: window size for sequence creation in hours. A sequence would contain a location for every 'window_hrs' 
        from start to end times
    country_for_missing: If True, the location for entries in the sequence when the individual wasn't in Italy would be
        set to the MCC code of the respective country
    n_threads: Number of threads to use in parallel
    """    

    # importing math for ceiling function
    import math
    from multiprocessing import Pool
    from itertools import repeat
    
    # finding the maximum time spent by an individual to set the length fo sequence
    max_time = max(df_trips['en_time'] - df_trips['st_time'])
    ncols = math.ceil(max_time.total_seconds()/(60*60*window_hrs))

    # If aligning by day of week of arrival, we need additional columns 
    # as someone could arrive on a sunday and stay for max_time
    if align_by_day_of_week == True:
        ncols += 6*math.ceil(24/window_hrs)

    # weeks = 10*np.linspace(1,6,6)
    # days= np.linspace(0,6.5,14)
    # columns = np.add.outer(weeks,days).flatten()
    
    # Initialising the sequence dataframe with NAs
    columns = np.linspace(1,ncols,ncols)
    cus_nr = df_trips['customer_nr']
    df_sequence = pd.DataFrame(np.nan,columns=columns,index=cus_nr)

    # Looping through every customer array to create the aligned sequences data frame
    p = Pool(n_threads)
    l = [i for i in range(0,len(cus_nr))]
    sequences_as_lists = p.starmap(create_sequence_for_individual, zip(l, [align_by_day_of_week]*len(cus_nr), [window_hrs]*len(cus_nr), [country_for_missing]*len(cus_nr), [ncols+1]*len(cus_nr)))

    return sequences_as_lists

In [ ]:
sd_seq1 = create_sequences(df_trips,align_by_day_of_week=True,window_hrs=3,country_for_missing=True,n_threads=8)

In [ ]:
sd_seq1

In [ ]:
max_time = max(df_trips['en_time'] - df_trips['st_time'])
ncols = math.ceil(max_time.total_seconds()/(60*60*window_hrs))

# If aligning by day of week of arrival, we need additional columns 
# as someone could arrive on a sunday and stay for max_time
if align_by_day_of_week == True:
    ncols += 6*math.ceil(24/window_hrs)

In [ ]:
create_sequence_for_individual(1,align_by_day_of_week=True,window_hrs=12,country_for_missing=True,ncols=ncols+1)

In [ ]:
seq = [np.nan] * (ncols+1)

In [ ]:
seq[0] = 1

In [ ]:
seq[1:] = [2]*ncols

In [ ]:
seq

In [ ]:
sd_seq1

In [ ]:
import py_common_subseq

In [ ]:
df_seq = pd.read_csv("/mnt/data/kmohan/TPT_tourism/new_codebase/src/models/sequence_analysis/data/sequences/sequences_chinese_pre-summer.csv")

In [ ]:
df_seq.head()